# Naive Bayes Classifier

In [211]:
import csv
import random
import math
import numpy as np
import pandas as pd
from IPython.display import display

In [212]:
def loadCsv(filename):
    # Numpy function to generate array from txt or csv
    return np.genfromtxt(filename, delimiter = '\t')

In [213]:
def splitDataset(dataset, splitRatio, number_of_rows):
    # Shuffled data
    data_copy = dataset.copy()
    np.random.shuffle(data_copy)
    
    # Cut dataset:
    data_part = data_copy[:int(number_of_rows)]
    
    # Training set size 
    trainSize = int(len(data_part) * splitRatio)
    
    #Split data
    data_train = data_part[: trainSize]
    data_test = data_part[trainSize : dataset.shape[0]]
    
    return data_train, data_test

In [214]:
def separateByClass(dataset):
    # Here we limit our classes to 0 and 1
    # You need to generalize this for arbitrary number of classes
    
    classes = np.unique(dataset[:, -1])
    return dict((k, dataset[np.where(dataset[:, -1] == k), :]) for k in classes)

   

In [215]:
def summarize(dataset):
    # Calculate means and standart deviations with one degree of freedom for each attribute
    # We do it by column which is axis 1
    # Also we remove last elements (guess why?)
    means = dataset.mean(axis = 1)[0][:-1]
    stds = dataset.std(axis = 1, ddof = 1)[0][:-1]
    
    # Think what we do here?
    return means, stds

In [216]:
def summarizeByClass(dataset):
    # Divide dataset by class and summarize it
    separated = separateByClass(dataset)
    
    summaries = {}
    
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    
    return summaries

In [217]:
def calculateProbability(x, mean, stdev):
    # Calculate probability by x, mean and std
    # 1/(sqrt(2pi)*std)*exp(-(x-mean)^2/(2std^2))
    return np.prod(np.exp(-(x-mean)**2/(2*stdev**2))/(np.sqrt(2*np.pi)*stdev))

In [218]:
def calculateClassProbabilities(summaries, inputVector):
    # Calculate probabilities for input vector from test set
    probabilities = {}
    
    for classValue, classSummaries in summaries.items():
        
        means = classSummaries[0]
        stds  = classSummaries[1]
        
        # Calculate corresonding probabilities and multiply them
        probabilities[classValue] = calculateProbability(inputVector[:-1], means, stds)
        
    return probabilities

In [219]:
def predict(summaries, inputVector):
    # Calculate probabilities
    probabilities = calculateClassProbabilities(summaries, inputVector)
    
    # Init values of probability and label
    bestLabel, bestProb = None, -1
    
    # Check probability of which class is better
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    
    return bestLabel

In [220]:
def getPredictions(summaries, testSet):
    # For each probability find optimal labels
    predictions = []
    
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)

    return predictions

In [221]:
def getAccuracy(testSet, predictions):
    # Check accuracy
    correct = 0
    
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [222]:
def main():
    # Set initial data
    filename = 'Skin_NonSkin.txt'
    
    # Set split ratio
    splitRatio = 0.67
    
    # Set number of rows (the dataset is too big)
    number_of_rows = 500
    
    # Load dataset and return numpy array
    dataset = loadCsv(filename)
    
    # Split dataset
    trainingSet, testSet = splitDataset(dataset, splitRatio, number_of_rows)
    
    # Log row amounts
    print('Split {0} rows into train = {1} and test = {2} rows'.format(number_of_rows, len(trainingSet), len(testSet)))
    
    # Prepare model
    summaries = summarizeByClass(trainingSet)
    
    # Test model
    predictions = getPredictions(summaries, testSet)
    
    accuracy = round(getAccuracy(testSet, predictions), 3)
    
    print('Accuracy: {0}%'.format(accuracy))
    
    # read data
    data = pd.DataFrame(testSet)
    data.columns = ["B", "G", "R", "Skin/No skin"]
    data.head(5)
    
    # Create a list to store the data
    predict_class = []
    
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predict_class.append(result)
        
    # Create a column from the list
    data['Predictions'] = predict_class
    display(data)
    

In [223]:
main()

Split 500 rows into train = 335 and test = 165 rows
Accuracy: 93.333%


B      G      R  Skin/No skin  Predictions
0    183.0  181.0  133.0           2.0          2.0
1     94.0   93.0   59.0           2.0          2.0
2     76.0   78.0   36.0           2.0          2.0
3    154.0  160.0  119.0           2.0          2.0
4      9.0    6.0    0.0           2.0          2.0
5    146.0  157.0  209.0           2.0          1.0
6    149.0  152.0  103.0           2.0          2.0
7    107.0  145.0  209.0           1.0          1.0
8    204.0  202.0  171.0           2.0          2.0
9     80.0   81.0   61.0           2.0          2.0
10   135.0  175.0  223.0           1.0          1.0
11    53.0   57.0   22.0           2.0          2.0
12    73.0   77.0   25.0           2.0          2.0
13   105.0  108.0   59.0           2.0          2.0
14   190.0  188.0  147.0           2.0          2.0
15   180.0  175.0  130.0           2.0          2.0
16   168.0  169.0  119.0           2.0          2.0
17   110.0  110.0  110.0           2.0          2.0
18    65.0   66.0   46.0           2.0          2.0
19   157.0  160.0  121.0           2.0          2.0
20   175.0  174.0  124.0           2.0          2.0
21   183.0  180.0  136.0           2.0          2.0
22   172.0   67.0   94.0           2.0          2.0
23   203.0  201.0  167.0           2.0          2.0
24   179.0  175.0  134.0           2.0          2.0
25    54.0   55.0   21.0           2.0          2.0
26    54.0   96.0  143.0           1.0          2.0
27    39.0   41.0   29.0           2.0          2.0
28    55.0   56.0   24.0           2.0          2.0
29   246.0  251.0  250.0           2.0          2.0
..     ...    ...    ...           ...          ...
135   15.0   13.0    2.0           2.0          2.0
136  109.0  149.0  208.0           1.0          1.0
137  200.0  198.0  163.0           2.0          2.0
138   71.0   30.0  245.0           2.0          2.0
139  161.0  154.0  167.0           2.0          1.0
140  127.0   65.0   55.0           2.0          2.0
141   70.0   71.0   29.0           2.0          2.0
142  146.0  147.0   97.0           2.0          2.0
143  152.0  153.0  103.0           2.0          2.0
144   84.0   84.0   36.0           2.0          2.0
145  183.0  177.0  134.0           2.0          2.0
146  178.0  175.0  131.0           2.0          2.0
147  169.0  166.0  115.0           2.0          2.0
148   54.0   62.0   22.0           2.0          2.0
149   21.0   15.0    2.0           2.0          2.0
150   67.0   70.0   31.0           2.0          2.0
151  198.0  197.0  159.0           2.0          2.0
152   95.0  149.0  220.0           1.0          1.0
153  136.0  139.0   90.0           2.0          2.0
154    3.0    2.0    0.0           2.0          2.0
155  174.0  172.0  124.0           2.0          2.0
156  141.0  140.0   82.0           2.0          2.0
157   96.0  140.0  217.0           1.0          1.0
158  181.0  177.0  129.0           2.0          2.0
159  178.0  178.0  138.0           2.0          2.0
160  198.0  206.0  253.0           1.0          1.0
161  197.0  196.0  158.0           2.0          2.0
162  138.0  139.0   89.0           2.0          2.0
163   68.0   69.0   25.0           2.0          2.0
164   67.0   68.0   24.0           2.0          2.0

[165 rows x 5 columns]